### FILE EXTRACT

In [17]:
import glob                         # Manipulation de fichiers
import pandas as pd                 # Manipulation de données
import xml.etree.ElementTree as ET  # Manipulation de fichier xlm
from datetime import datetime       # gestion de temps

# Fonctions permettant d’extraire les données de différents formats de fichiers.
# .csv .json .xml .xlsx

source_fles = r"..\source_files"         # tous les fichiers locals

# Extraction de fichier csv
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process)
    return dataframe

# Extraction de fichier .xlsx
def extract_from_excel(file_to_process):
    dataframe = pd.read_excel(file_to_process)
    return dataframe

# Extraction de fichier .json
def extract_from_json(file_to_process): 
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

# Extraction de fichier .xlm
def extract_from_xml(file_to_process): 
    dataframe = pd.DataFrame(columns=["name", "height", "weight"]) 
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    for person in root: 
        name = person.find("name").text 
        height = float(person.find("height").text) 
        weight = float(person.find("weight").text) 
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True) 
    return dataframe

# Extraction de tous les fichiers



In [18]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data     
    # process all csv files 
    for csvfile in glob.glob(f"{source_fles}/*.csv"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)         
    # process all json files 
    for jsonfile in glob.glob(f"{source_fles}/*.json"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True)     
    # process all xml files 
    for xmlfile in glob.glob(f"{source_fles}/*.xml"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True)         
    return extracted_data

In [16]:
source_fles = r"..\source_files"
glob.glob(f"{source_fles}/*.csv")

['..\\source_files\\source1.csv',
 '..\\source_files\\source2.csv',
 '..\\source_files\\source3.csv']

### WEB SCRAPING

In [5]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup as bs

In [38]:
url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
db_name = r'../data/movies.db'
table_name = 'top_film'
csv_path = r'../data/top_film.csv'
df = pd.DataFrame(columns=["average_rank","film","year"])
count = 0

In [6]:
r = requests.get(url)                            # GET HTTS PAGE
html_page = r.text                               # CONVET HTTP TO TEXT
soup = bs(html_page, 'html.parser')

In [50]:
tables = soup.find_all('tbody')
rows = tables[0].find_all('tr')

for row in rows:
    col = row.find_all('td')
    if len(col)!=0:
        data_dict = {"average_rank": col[0].contents[0],
                        "film": col[1].contents[0],
                        "year": col[2].contents[0]}
        df1 = pd.DataFrame(data_dict, index=[0])
        df = pd.concat([df,df1], ignore_index=True)
        count+=1

In [51]:
db_name = '../data/movies.db'
table_name = 'top_film'

conn = sqlite3.connect(db_name)
df.to_sql(table_name, conn, if_exists='replace', index=False)

# Exécuter une requête SQL pour sélectionner toutes les données de la table
query = f"SELECT * FROM {table_name}"
df_result = pd.read_sql_query(query, conn)

In [55]:
query = f"SELECT * FROM {table_name}"
df_result = pd.read_sql_query(query, conn)


In [47]:
import sqlite3
import pandas as pd

# Nom de la base de données et de la table
db_name = '../data/movies222.db'
table_name = 'top_film'

# Se connecter à la base de données SQLite
conn = sqlite3.connect(db_name)

# Exemple de DataFrame à insérer dans la base de données
data = {'title': ['Inception', 'Interstellar', 'The Dark Knight'],
        'director': ['Christopher Nolan', 'Christopher Nolan', 'Christopher Nolan'],
        'year': [2010, 2014, 2008]}
df = pd.DataFrame(data)

# Insérer le DataFrame dans une table SQLite
df.to_sql(table_name, conn, if_exists='replace', index=False)

# Exécuter une requête SQL pour sélectionner toutes les données de la table
query = f"SELECT * FROM {table_name}"
df_result = pd.read_sql_query(query, conn)

# Afficher les résultats de la requête
print(df_result)

# Fermer la connexion à la base de données
conn.close()


             title           director  year
0        Inception  Christopher Nolan  2010
1     Interstellar  Christopher Nolan  2014
2  The Dark Knight  Christopher Nolan  2008


In [48]:
df

,title,director,year
0,Inception,Christopher Nolan,2010
1,Interstellar,Christopher Nolan,2014
2,The Dark Knight,Christopher Nolan,2008


In [49]:
df_result

,title,director,year
0,Inception,Christopher Nolan,2010
1,Interstellar,Christopher Nolan,2014
2,The Dark Knight,Christopher Nolan,2008
